In [ ]:
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
from google.colab import files
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from collections import deque


In [ ]:
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
import os
from google.colab import files
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from collections import deque

class HandGestureAnalyzer:
    def __init__(self):
        # Initialize MediaPipe
        self.mp_hands = mp.solutions.hands
        self.mp_pose = mp.solutions.pose
        self.mp_face_mesh = mp.solutions.face_mesh

        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # Initialize tracking variables
        self.reset_tracking()

    def reset_tracking(self):
        """Reset all cumulative tracking variables"""
        self.cumulative_gesture_frequency = 0
        self.cumulative_face_touches = 0
        self.cumulative_smoothness = 0.0

        # History for smoothness calculation
        self.hand_history = {
            'left': deque(maxlen=5),   # Last 5 positions for smoothness
            'right': deque(maxlen=5)
        }

        # Previous positions for velocity calculation
        self.prev_hand_positions = {'left': None, 'right': None}

        # Gesture detection state
        self.prev_gesture_state = None

        # Face region cache
        self.face_region = None

    def is_black_frame(self, frame, threshold=15):
        mean_brightness = np.mean(frame)
        return mean_brightness < threshold

    def extract_hand_landmarks(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)

        hand_landmarks = {'left': None, 'right': None}

        if results.multi_hand_landmarks:
            for idx, hand_lms in enumerate(results.multi_hand_landmarks):
                # Get hand label (left/right)
                if results.multi_handedness:
                    hand_label = results.multi_handedness[idx].classification[0].label.lower()
                else:
                    hand_label = 'right' if idx == 0 else 'left' # Fallback

                # Extract landmark coordinates
                h, w = frame.shape[:2]
                landmarks = []
                for lm in hand_lms.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    landmarks.append([x, y])

                hand_landmarks[hand_label] = np.array(landmarks)

        return hand_landmarks

    def extract_face_region(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0]
            h, w = frame.shape[:2]

            # Get face boundary points
            face_points = []
            for lm in landmarks.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                face_points.append([x, y])

            face_points = np.array(face_points)

            # Calculate face bounding box with margin
            margin = 50  # pixels
            min_x = max(0, np.min(face_points[:, 0]) - margin)
            max_x = min(w, np.max(face_points[:, 0]) + margin)
            min_y = max(0, np.min(face_points[:, 1]) - margin)
            max_y = min(h, np.max(face_points[:, 1]) + margin)

            return {
                'min_x': min_x, 'max_x': max_x,
                'min_y': min_y, 'max_y': max_y,
                'center': [(min_x + max_x) // 2, (min_y + max_y) // 2]
            }

        return None

    def calculate_hand_velocity(self, current_positions, fps):
        velocities = {'left': 0.0, 'right': 0.0}

        for hand in ['left', 'right']:
            if (current_positions[hand] is not None and
                self.prev_hand_positions[hand] is not None):

                current_center = np.mean(current_positions[hand], axis=0)
                prev_center = np.mean(self.prev_hand_positions[hand], axis=0)

                distance = euclidean(current_center, prev_center)
                time_delta = 1.0 / fps if fps > 0 else 1.0/30.0
                velocity = distance / time_delta
                velocities[hand] = velocity

        self.prev_hand_positions = current_positions.copy()
        return velocities

    def detect_gesture_change(self, hand_landmarks):
        if not any(lm is not None for lm in hand_landmarks.values()):
            current_state = "no_hands"
        else:
            gesture_features = []
            for hand, landmarks in hand_landmarks.items():
                if landmarks is not None:
                    # Fingertips indices: 4, 8, 12, 16, 20
                    fingertips = [4, 8, 12, 16, 20]
                    if len(landmarks) > max(fingertips):
                        tip_distances = []
                        for i in range(len(fingertips)-1):
                            for j in range(i+1, len(fingertips)):
                                dist = euclidean(landmarks[fingertips[i]], landmarks[fingertips[j]])
                                tip_distances.append(dist)
                        avg_tip_distance = np.mean(tip_distances) if tip_distances else 0
                        gesture_features.append(avg_tip_distance)

            if not gesture_features:
                current_state = "no_hands"
            elif np.mean(gesture_features) > 100:
                current_state = "open_hand"
            else:
                current_state = "closed_hand"

        gesture_changed = (self.prev_gesture_state is not None and
                          current_state != self.prev_gesture_state)

        if gesture_changed:
            self.cumulative_gesture_frequency += 1

        self.prev_gesture_state = current_state
        return gesture_changed

    def detect_face_touch(self, hand_landmarks, face_region):
        if face_region is None:
            return False

        face_touch = False
        for hand, landmarks in hand_landmarks.items():
            if landmarks is not None:
                for point in landmarks:
                    x, y = point
                    if (face_region['min_x'] <= x <= face_region['max_x'] and
                        face_region['min_y'] <= y <= face_region['max_y']):
                        face_touch = True
                        break
                if face_touch: break

        if face_touch:
            self.cumulative_face_touches += 1
        return face_touch

    def calculate_smoothness(self, hand_landmarks):
        smoothness_scores = {'left': 1.0, 'right': 1.0}

        for hand in ['left', 'right']:
            if hand_landmarks[hand] is not None:
                hand_center = np.mean(hand_landmarks[hand], axis=0)
                self.hand_history[hand].append(hand_center)

                if len(self.hand_history[hand]) >= 4:
                    positions = list(self.hand_history[hand])
                    velocities = []
                    for i in range(1, len(positions)):
                        vel = euclidean(positions[i], positions[i-1])
                        velocities.append(vel)
                    
                    accelerations = []
                    for i in range(1, len(velocities)):
                        acc = abs(velocities[i] - velocities[i-1])
                        accelerations.append(acc)

                    if len(accelerations) >= 2:
                        jerks = []
                        for i in range(1, len(accelerations)):
                            jerk = abs(accelerations[i] - accelerations[i-1])
                            jerks.append(jerk)

                        avg_jerk = np.mean(jerks)
                        smoothness = 1.0 / (1.0 + avg_jerk / 10.0)
                        smoothness_scores[hand] = smoothness

        current_smoothness = np.mean(list(smoothness_scores.values()))
        if self.cumulative_smoothness == 0:
            self.cumulative_smoothness = current_smoothness
        else:
            self.cumulative_smoothness = 0.9 * self.cumulative_smoothness + 0.1 * current_smoothness

        return smoothness_scores

    def get_neutral_hand_data(self, frame_count, fps):
        return {
            'frame': frame_count,
            'timestamp': frame_count / fps,
            'left_hand_velocity': 0.0, 'right_hand_velocity': 0.0, 'avg_hand_velocity': 0.0,
            'gesture_frequency_cumulative': self.cumulative_gesture_frequency,
            'face_touches_cumulative': self.cumulative_face_touches,
            'smoothness_cumulative': self.cumulative_smoothness,
            'current_smoothness_left': 1.0, 'current_smoothness_right': 1.0,
            'hands_detected': False, 'face_detected': False,
            'is_black_frame': True, 'gesture_change': False, 'face_touch': False
        }

    def process_frame(self, frame, frame_count, fps):
        if self.is_black_frame(frame):
            return self.get_neutral_hand_data(frame_count, fps)

        hand_landmarks = self.extract_hand_landmarks(frame)
        hands_detected = any(lm is not None for lm in hand_landmarks.values())
        
        face_region = self.extract_face_region(frame)
        face_detected = face_region is not None

        if not hands_detected:
            data = self.get_neutral_hand_data(frame_count, fps)
            data['is_black_frame'] = False
            data['face_detected'] = face_detected
            return data

        velocities = self.calculate_hand_velocity(hand_landmarks, fps)
        avg_velocity = np.mean([v for v in velocities.values() if v > 0]) if any(velocities.values()) else 0.0
        
        gesture_change = self.detect_gesture_change(hand_landmarks)
        face_touch = self.detect_face_touch(hand_landmarks, face_region)
        smoothness_scores = self.calculate_smoothness(hand_landmarks)

        return {
            'frame': frame_count,
            'timestamp': frame_count / fps,
            'left_hand_velocity': velocities['left'],
            'right_hand_velocity': velocities['right'],
            'avg_hand_velocity': avg_velocity,
            'gesture_frequency_cumulative': self.cumulative_gesture_frequency,
            'face_touches_cumulative': self.cumulative_face_touches,
            'smoothness_cumulative': self.cumulative_smoothness,
            'current_smoothness_left': smoothness_scores['left'],
            'current_smoothness_right': smoothness_scores['right'],
            'hands_detected': hands_detected,
            'face_detected': face_detected,
            'is_black_frame': False,
            'gesture_change': gesture_change,
            'face_touch': face_touch
        }

def process_hand_gestures(video_path):
    """
    Main function to process video and download results with dynamic filenames.
    """
    if not os.path.exists(video_path):
        print(f"❌ Error: File not found at {video_path}")
        return

    # --- Setup Dynamic Filenames ---
    base_name = os.path.basename(video_path)
    file_name_only = os.path.splitext(base_name)[0]
    
    # Define output names based on input video name
    # csv_complete = f"{file_name_only}.csv"
    csv_hands_only = f"{file_name_only}.csv"
    # csv_summary = f"{file_name_only}_summary.csv"

    print(f"🎬 Processing: {base_name}")
    print(f"📂 Main Output will be: {csv_hands_only}")

    # --- Processing Loop ---
    analyzer = HandGestureAnalyzer()
    cap = cv2.VideoCapture(video_path)
    results = []
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0 or np.isnan(fps): fps = 30.0
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process every 10th frame
        if frame_count % 10 == 0:
            frame_data = analyzer.process_frame(frame, frame_count, fps)
            results.append(frame_data)

        frame_count += 1
        if frame_count % 500 == 0:
            print(f"  ...processed {frame_count} frames")

    cap.release()
    df = pd.DataFrame(results)

    if len(df) == 0:
        print("⚠️ No data extracted.")
        return

    # --- Saving and Downloading ---
    print(f"\n💾 Saving results...")

    # 1. Complete dataset
    # df.to_csv(csv_complete, index=False)
    # files.download(csv_complete)

    # 2. Hands-only dataset
    hands_only_df = df[df['hands_detected'] == True].copy()
    if len(hands_only_df) > 0:
        hands_only_df.to_csv(csv_hands_only, index=False)
        files.download(csv_hands_only)

    print(f"\nAll files downloaded for {base_name}")

In [ ]:

# Main processing function
def process_video_hand_gestures(video_path):
    """Process video for hand gesture analysis"""

    analyzer = HandGestureAnalyzer()
    cap = cv2.VideoCapture(video_path)
    results = []

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_count = 0

    print("Processing video for hand gesture analysis...")
    print(f"Video: {fps:.1f} FPS, {total_frames} total frames")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process every 10th frame for reasonable performance
        if frame_count % 10 == 0:
            frame_data = analyzer.process_frame(frame, frame_count, fps)
            results.append(frame_data)

        frame_count += 1
        if frame_count % 300 == 0:  # Progress update every 300 frames
            print(f"Processed {frame_count}/{total_frames} frames "
                  f"({frame_count/total_frames*100:.1f}%)")

    cap.release()
    return pd.DataFrame(results)


In [ ]:

# Upload and process video
print("Upload your video for hand gesture analysis:")
# uploaded = files.upload()
# video_path = list(uploaded.keys())[0]

# Process video
df = process_video_hand_gestures("/content/Stephen_on_StephenKeala.mov")

print(f"\nSuccessfully processed {len(df)} frames!")

# Show statistics
hands_frames = df['hands_detected'].sum()
face_frames = df['face_detected'].sum()
black_frames = df['is_black_frame'].sum()
total_gestures = df['gesture_frequency_cumulative'].iloc[-1] if len(df) > 0 else 0
total_face_touches = df['face_touches_cumulative'].iloc[-1] if len(df) > 0 else 0

print(f"\nAnalysis Summary:")
print(f"Frames with hands detected: {hands_frames}")
print(f"Frames with face detected: {face_frames}")
print(f"Black frames: {black_frames}")
print(f"Total gesture changes: {total_gestures}")
print(f"Total face touches: {total_face_touches}")
print(f"Final smoothness score: {df['smoothness_cumulative'].iloc[-1]:.3f}" if len(df) > 0 else "N/A")

# Show sample data
print(f"\nSample hand gesture data:")
sample_cols = ['frame', 'timestamp', 'avg_hand_velocity', 'gesture_frequency_cumulative',
               'face_touches_cumulative', 'smoothness_cumulative', 'hands_detected']
print(df[sample_cols].head(10))

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Hand velocities over time
ax1 = axes[0, 0]
ax1.plot(df['timestamp'], df['left_hand_velocity'], label='Left Hand', alpha=0.7)
ax1.plot(df['timestamp'], df['right_hand_velocity'], label='Right Hand', alpha=0.7)
ax1.plot(df['timestamp'], df['avg_hand_velocity'], label='Average', linewidth=2)
ax1.set_title('Hand Velocity Over Time')
ax1.set_xlabel('Time (seconds)')
ax1.set_ylabel('Velocity (pixels/second)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Cumulative gesture frequency
ax2 = axes[0, 1]
ax2.plot(df['timestamp'], df['gesture_frequency_cumulative'], color='green', linewidth=2)
ax2.set_title('Cumulative Gesture Changes')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Total Gesture Changes')
ax2.grid(True, alpha=0.3)

# Plot 3: Cumulative face touches
ax3 = axes[1, 0]
ax3.plot(df['timestamp'], df['face_touches_cumulative'], color='red', linewidth=2)
ax3.set_title('Cumulative Face Touches')
ax3.set_xlabel('Time (seconds)')
ax3.set_ylabel('Total Face Touches')
ax3.grid(True, alpha=0.3)

# Plot 4: Movement smoothness
ax4 = axes[1, 1]
ax4.plot(df['timestamp'], df['smoothness_cumulative'], color='purple', linewidth=2)
ax4.plot(df['timestamp'], df['current_smoothness_left'], alpha=0.5, label='Left Hand Current')
ax4.plot(df['timestamp'], df['current_smoothness_right'], alpha=0.5, label='Right Hand Current')
ax4.set_title('Movement Smoothness')
ax4.set_xlabel('Time (seconds)')
ax4.set_ylabel('Smoothness Score (0-1)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Additional analysis plots
plt.figure(figsize=(15, 8))

# Velocity distribution
plt.subplot(2, 3, 1)
velocities = df[df['hands_detected'] == True]['avg_hand_velocity']
plt.hist(velocities, bins=30, alpha=0.7, color='blue')
plt.title('Hand Velocity Distribution')
plt.xlabel('Velocity (pixels/second)')
plt.ylabel('Frequency')

# Gesture frequency rate
plt.subplot(2, 3, 2)
gesture_rate = df['gesture_frequency_cumulative'].diff().fillna(0)
plt.plot(df['timestamp'], gesture_rate, color='green')
plt.title('Gesture Change Rate')
plt.xlabel('Time (seconds)')
plt.ylabel('Gestures per frame')

# Face touch events
plt.subplot(2, 3, 3)
touch_events = df['face_touch'].astype(int)
plt.plot(df['timestamp'], touch_events, 'ro', markersize=3, alpha=0.7)
plt.title('Face Touch Events')
plt.xlabel('Time (seconds)')
plt.ylabel('Touch Detected')

# Hands detection timeline
plt.subplot(2, 3, 4)
hands_timeline = df['hands_detected'].astype(int)
plt.plot(df['timestamp'], hands_timeline, color='orange', linewidth=2)
plt.title('Hands Detection Timeline')
plt.xlabel('Time (seconds)')
plt.ylabel('Hands Detected')

# Smoothness comparison
plt.subplot(2, 3, 5)
plt.plot(df['timestamp'], df['current_smoothness_left'], label='Left Hand')
plt.plot(df['timestamp'], df['current_smoothness_right'], label='Right Hand')
plt.title('Per-Hand Smoothness')
plt.xlabel('Time (seconds)')
plt.ylabel('Smoothness Score')
plt.legend()

# Combined metrics
plt.subplot(2, 3, 6)
# Normalize metrics for comparison
norm_velocity = df['avg_hand_velocity'] / df['avg_hand_velocity'].max() if df['avg_hand_velocity'].max() > 0 else df['avg_hand_velocity']
norm_gestures = df['gesture_frequency_cumulative'] / df['gesture_frequency_cumulative'].max() if df['gesture_frequency_cumulative'].max() > 0 else df['gesture_frequency_cumulative']
norm_touches = df['face_touches_cumulative'] / df['face_touches_cumulative'].max() if df['face_touches_cumulative'].max() > 0 else df['face_touches_cumulative']

plt.plot(df['timestamp'], norm_velocity, label='Velocity (norm)', alpha=0.7)
plt.plot(df['timestamp'], norm_gestures, label='Gestures (norm)', alpha=0.7)
plt.plot(df['timestamp'], norm_touches, label='Touches (norm)', alpha=0.7)
plt.plot(df['timestamp'], df['smoothness_cumulative'], label='Smoothness', alpha=0.7)
plt.title('Normalized Metrics Comparison')
plt.xlabel('Time (seconds)')
plt.ylabel('Normalized Value (0-1)')
plt.legend()

plt.tight_layout()
plt.show()

# Save results
print(f"\nSaving results...")

# Complete dataset
df.to_csv('hand_gesture_analysis_complete.csv', index=False)
files.download('hand_gesture_analysis_complete.csv')

# Hands-only dataset
hands_only_df = df[df['hands_detected'] == True].copy()
if len(hands_only_df) > 0:
    hands_only_df.to_csv('hand_gesture_analysis_hands_only.csv', index=False)
    files.download('hand_gesture_analysis_hands_only.csv')
    print(f"Hands-only dataset: {len(hands_only_df)} frames")

# Summary statistics
summary_stats = {
    'total_frames_processed': len(df),
    'frames_with_hands': hands_frames,
    'frames_with_face': face_frames,
    'black_frames': black_frames,
    'total_gesture_changes': total_gestures,
    'total_face_touches': total_face_touches,
    'final_smoothness_score': df['smoothness_cumulative'].iloc[-1] if len(df) > 0 else 0,
    'avg_hand_velocity': df[df['hands_detected'] == True]['avg_hand_velocity'].mean() if hands_frames > 0 else 0,
    'max_hand_velocity': df['avg_hand_velocity'].max(),
    'video_duration_seconds': df['timestamp'].max() if len(df) > 0 else 0
}

summary_df = pd.DataFrame([summary_stats])
summary_df.to_csv('hand_gesture_summary.csv', index=False)
files.download('hand_gesture_summary.csv')

print("Hand gesture analysis complete!")
print("\nFiles downloaded:")
print("  1. hand_gesture_analysis_complete.csv - All frames with hand data")
print("  2. hand_gesture_analysis_hands_only.csv - Only frames with detected hands")
print("  3. hand_gesture_summary.csv - Summary statistics")

print(f"\nFinal Results:")
print(f"  • Average hand velocity: {summary_stats['avg_hand_velocity']:.1f} pixels/second")
print(f"  • Total gesture changes: {summary_stats['total_gesture_changes']}")
print(f"  • Total face touches: {summary_stats['total_face_touches']}")
print(f"  • Final smoothness score: {summary_stats['final_smoothness_score']:.3f}")

In [ ]:
import os

def process_all_videos_in_folder(folder_path):
    """
    Finds all .mov files in the folder and applies process_hand_gestures to them.
    """
    # Verify folder exists
    if not os.path.exists(folder_path):
        print(f"❌ Error: Folder not found at {folder_path}")
        return

    # Get list of all files
    all_files = os.listdir(folder_path)
    
    # Filter for .mov files (case insensitive)
    mov_files = [f for f in all_files if f.lower().endswith('.mov')]
    
    total_files = len(mov_files)
    print(f"📂 Found {total_files} video files in folder.\n")

    # Loop through each file
    for index, filename in enumerate(mov_files):
        video_path = os.path.join(folder_path, filename)
        
        print(f"--------------------------------------------------")
        print(f"▶️ [{index + 1}/{total_files}] Starting: {filename}")
        
        try:
            # Call the function you defined previously
            process_hand_gestures(video_path)
            print(f"✅ Completed: {filename}")
        
        except Exception as e:
            print(f"⚠️ Failed to process {filename}")
            print(f"Error details: {e}")

    print(f"\n🎉 Batch processing finished!")

# --- CONFIGURATION ---
# Change this path to your folder
target_folder = "/content/drive/MyDrive/DatasetCercetare/VideosEdited/"

# Run the batch
process_all_videos_in_folder(target_folder)